# Challenge 6: Actions
## Introduction

In this part of the challenge you will add another agent to the solution.

This time will be an agent that performs actions on behalf of the user.

## Step 1: Setup the final enviroment

## Step 2: Build on top of the previous solution

### Import the necessary libraries

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

## Import the necessary libraries

import requests
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from urllib.parse import quote_plus 
from its_a_rag import ingestion
from sqlalchemy import create_engine
from langchain.agents import AgentExecutor, create_sql_agent, create_openai_tools_agent
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.tools import tool

### Create Environment Variables

**Important:** Make sure you update your `.env` file.

In [2]:
import os, dotenv, sys
dotenv.load_dotenv(override=True)

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))


# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")

# Local Folder for the documents
LOCAL_FOLDER = "../../data/fsi/pdf"

# SQL Database
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")

# STOCK API
MOCK_API_ENDPOINT= os.getenv("MOCK_API_ENDPOINT")

# Define the questions list (if you are using your own dataset you need to change this list)
QUESTIONS = [
  "What are the revenues of GOOGLE in the year 2009?",
  "What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?",
  "Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?",
  "Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?",
  "Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.",
  "What is the value of the cumulative 5-years total return of ALPHABET Class A at December 2022?",
  "What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?",
  "Can you buy 10 shares of APPLE for me?"
  ]

# Define the System prompt (you need to update this is you are using your own dataset.)
system_prompt_RAG = """ You are a financial assistant tasked with answering questions related to the financial results of major technology companies listed on NASDAQ, \n
specifically Microsoft (MSFT), Alphabet Inc. (GOOGL), Nvidia (NVDA), Apple Inc. (AAPL), and Amazon (AMZN). \n
if you don't find the answer in the context, just say `I don't know.`"""

system_prompt_START = """
  You are an agent that needs analyze the user question. \n
  Question : {input} \n
  if the question is related to stock prices answer with "stock". \n
  if the question is related to information about financial results answer with "rag". \n
  if the question is a direct requesto for trading, buying or selling stocks answer with "trade". \n
  if the question is unclear or you cannot decide answer with "rag". \n
  only answer with one of the words provided.
  Your answer (stock/rag/trade):
  """
system_prompt_SQL = """
  You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices.
  If you can't find the answer, say 'I am unable to find the answer.'
  """

## Step 1: Build on top of the previous solution

In [3]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2
)

In [4]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [5]:
# Stock Agent

def stock_agent(state):
    from langchain.agents import AgentType
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm

    # Create the SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
    engine = create_engine(
        f"mssql+pymssql://{SQL_USERNAME}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DB}")

    db = SQLDatabase(engine=engine)
    stock_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    # Create the agent using the Langhcain SQL Agent Class (create_sql_agent)
    stock_agent = create_sql_agent(llm=llm,
                                   toolkit=stock_toolkit,
                                   agent_type="openai-tools",
                                   agent_name="StockAgent",
                                   agent_description="This agent is an expert in querying SQL Database to find answers to user's question about stock prices.",
                                   agent_version="0.1",
                                   agent_author="itsarag",
                                   # verbose=True,
                                   agent_executor_kwargs=dict(handle_parsing_errors=True))
    # Structure the final prompt from the ChatPromptTemplate
    sql_prompt = ChatPromptTemplate.from_messages(
        [("system", system_prompt_SQL),
         ("user", "{question}\n ai:")])
    # Prepare the response using the invoke method of the agent
    response = stock_agent.invoke(sql_prompt.format(
        question=state["input"]))
    # Return the response for the next agent (output and input required coming fron the Agent State)
    return {"output": response['output'], "input": state["input"]}


test_stock_state = AgentState(
    input=QUESTIONS[6], output="", decision="", messages=[])
result = stock_agent(test_stock_state)
print(result)

{'output': 'On 23/07/2024, the stock prices were as follows:\n- APPLE (AAPL): $225.01\n- NVIDIA (NVDA): $122.59\n- MICROSOFT (MSFT): $444.85', 'input': 'What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?'}


In [6]:
# Node rag (this is a clean implementation of the RAG Agent completed in Challenge 4)
def rag_agent(state):
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm
    rag_agent_llm = llm
    # Define the index (use the one created in the previous challenge)
    retriever_multimodal = AzureAISearchRetriever(
        index_name=AZURE_SEARCH_INDEX, api_key=AZURE_SEARCH_API_KEY, service_name=AZURE_SEARCH_ENDPOINT, top_k=5)
    # Define the chain (as it was in the previous challenge)
    chain_multimodal_rag = (
        {
            "context": retriever_multimodal | RunnableLambda(ingestion.get_image_description),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(ingestion.multimodal_prompt)
        | rag_agent_llm
        | StrOutputParser()
    )
    # prepare the response using the invoke method of the agent
    response = chain_multimodal_rag.invoke({"input": state["input"]})
    # Return the response for the next agent (output and input required coming from the Agent State)
    return {"output": response, "input": state["input"]}


print(QUESTIONS[4])
test_rag_state = AgentState(
    input=QUESTIONS[4], output="", decision="", messages=[])
result = rag_agent(test_rag_state)
print(result)

Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.
{'output': 'Yes, Apple repurchased common stock in 2023. Here is a table summarizing the repurchased stock:\n\n| Date       | Number of Stocks (in thousands) | Value (in dollars) |\n|------------|---------------------------------|---------------------|\n| 2023       | 471,419                         | $76.6 billion       |\n\nNote: The specific dates of the repurchases within 2023 are not provided in the context.', 'input': 'Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.'}


### Step 2: Create the Stock Action Agent

In [8]:
## Node stock_action
# Define the tool using the @tool decorator
@tool
def buy_stocks(quantity: int, symbol: str) -> str:
    """
    Permits to buy a quantity of shares of a specific stock.
    """
    if not symbol or not quantity:
        return f"Missing required arguments: symbol and quantity are required."
    try:
        response = requests.post(
            f"{MOCK_API_ENDPOINT}/api/buy/?stock_symbol={symbol}&quantity={quantity}")
        response.raise_for_status()
        return response.json()["message"]
    except requests.exceptions.RequestException as e:
        return f"Failed to buy stock. Exception: {str(e)}"


@tool
def sell_stocks(quantity: int, symbol: str) -> str:
    """
    Permits to sell a quantity of shares of a specific stock.
    """
    if not symbol or not quantity:
        return f"Missing required arguments: symbol and quantity are required."
    try:
        response = requests.post(
            f"{MOCK_API_ENDPOINT}/api/sell/?stock_symbol={symbol}&quantity={quantity}")
        response.raise_for_status()
        return response.json()["message"]
    except requests.exceptions.RequestException as e:
        return f"Failed to sell stock. Exception: {str(e)}"



# You can combine multiple tools in a single block called Toolkit
stock_toolkit = [buy_stocks, sell_stocks]


def stock_action(state):
    # Define the LLM
    global llm
    stock_action_llm = llm
    # Prepare the prompt for the agent (the create_openai_tools_agent function requires a ChatPromptTemplate object)
    # Prompt Example: "You are an agent that helps to acquire stocks. Use your tools to perform the requested action. \n if you can't perform the action, say 'I am unable to perform the action.' \n"
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
             """
            You are an agent that helps to acquire stocks. Use your tools to perform the requested action. \n
            If you can't perform the action, say 'I am unable to perform the action.'
            """
             ),
            MessagesPlaceholder("chat_history", optional=True),
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad"),
        ]
    )
    # Construct the OpenAI Tools Agent
    stock_buyer = create_openai_tools_agent(
        stock_action_llm, stock_toolkit, prompt)
    # Prepare the Agent Executor
    stock_buyer_executor = AgentExecutor(
        agent=stock_buyer, tools=stock_toolkit, verbose=True)
    # prepare the response using the invoke method of the agent
    response = stock_buyer_executor.invoke({"input": state["input"]})
    # Return the response for the next agent (output and input required coming from the Agent State)
    return {"output": response['output'], "input": state["input"]}

# Test the stock_action agent
print(QUESTIONS[7])
test_stock_action_state = AgentState(
    input=QUESTIONS[7], output="", decision="", messages=[])
result = stock_action(test_stock_action_state)
print(result)


    
    

Can you buy 10 shares of APPLE for me?


> Entering new AgentExecutor chain...

Invoking: `buy_stocks` with `{'quantity': 10, 'symbol': 'AAPL'}`


Bought 10 shares of AAPLI have successfully bought 10 shares of APPLE (AAPL) for you.

> Finished chain.
{'output': 'I have successfully bought 10 shares of APPLE (AAPL) for you.', 'input': 'Can you buy 10 shares of APPLE for me?'}


### Step 3: Update the Start Agent adding an addition decision answer

In [10]:
# Create the start_agent that analyze the user question and decide if the question is related to stock prices or financial results
def start_agent(state):
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm
    start_agent_llm = llm
    # Prepare the prompt for the agent
    # Prompt Example: You are an agent that needs analyze the user question. \n Question : {input} \n if the question is related to stock prices answer with "stock". \n if the question is related to information about financial results answer with "rag". \n if the question is unclear or you cannot decide answer with "rag". \n only answer with one of the word provided. Your answer (stock/rag):
    start_prompt = ChatPromptTemplate.from_messages(
        [("system", system_prompt_START)])

    # Prepare the chain to be executed
    chain = start_prompt | start_agent_llm
    # invoke the chain
    response = chain.invoke({"input": state["input"]})
    # take the decision from the response
    decision = response.content.strip().lower()
    # Return the response for the next agent (decision and input required coming fron the Agent State)
    return {"decision": decision, "input": state["input"]}


for QUESTION in QUESTIONS:
    print(QUESTION)
    test_start_state = AgentState(
        input=QUESTION, output="", decision="", messages=[])
    result = start_agent(test_start_state)
    print(result)
    print()

What are the revenues of GOOGLE in the year 2009?
{'decision': 'rag', 'input': 'What are the revenues of GOOGLE in the year 2009?'}

What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?
{'decision': 'rag', 'input': 'What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?'}

Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?
{'decision': 'rag', 'input': 'Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?'}

Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?
{'decision': 'rag', 'input': 'Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?'}

Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.
{'decision': 'rag', 'input': 'D

### Step 4: Update the previous graph to include the new agent

In [ ]:
# Create the 3 steps graph that is going to be working in the bellow "decision" condition
# Add nodes (start_agent, stock_agent, rag_agent) and conditional edges where the decision with be stock or rag
def create_graph():
    # Create the Workflow as StateGraph using the AgentState
    workflow = StateGraph(AgentState)
    # Add the nodes (start_agent, stock_agent, rag_agent)
    workflow.add_node("start", start_agent)
    workflow.add_node("stock_agent", stock_agent)
    workflow.add_node("trade_agent", stock_action)
    workflow.add_node("rag_agent", rag_agent)
    # Add the conditional edge from start -> lamba (decision) -> stock_agent or rag_agent
    workflow.add_conditional_edges(
        "start",
        lambda x: x["decision"],
        {
            "stock": "stock_agent",
            "rag": "rag_agent",
            "trade": "trade_agent"
        }
    )
    # Set the workflow entry point
    workflow.set_entry_point("start")
    # Add the final edges to the END node
    workflow.add_edge("stock_agent", END)
    workflow.add_edge("rag_agent", END)
    workflow.add_edge("trade_agent", END)
    # Compile the workflow
    return workflow.compile()

### Step 5: Test the Solution

In [ ]:
## Test Solution

# intantiate the graph (create_graph)
graph = create_graph()

# Use the graph invoke to answer the questions
# Test the graph with various questions

for QUESTION in QUESTIONS:
    print (QUESTION)
    result = graph.invoke({"input": QUESTION})
    print(result["output"])
    print ("------------------")